<a href="https://colab.research.google.com/github/ctotti/INPE_Disciplinas/blob/main/Masters_Thesis/AHP/Matriz_Comparacao_Pareada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from itertools import product
from tqdm import tqdm
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import get_column_letter

# 1. DEFINIÇÃO INICIAL E FUNÇÕES AUXILIARES -----------------------------------

# Ordem de importância desejada (do mais importante para o menos)
ordem_importancia = [
    "Uso e Cobertura da Terra",
    "Distância à Estradas",
    "Concentração de Estabelecimentos",
    "Textura do Solo",
    "Tamanho de Imóveis",
    "Declividade"
]

def construir_matriz(valores):
    """Constroi a matriz de comparação a partir dos valores superiores à diagonal"""
    # Índices triangulares superiores (sem a diagonal)
    triu_indices = [(0,1), (0,2), (0,3), (0,4), (0,5),
                   (1,2), (1,3), (1,4), (1,5),
                   (2,3), (2,4), (2,5),
                   (3,4), (3,5),
                   (4,5)]

    mat = np.identity(6)
    for (i,j), val in zip(triu_indices, valores):
        mat[i,j] = val
        mat[j,i] = 1/val
    return mat

def calcular_consistencia(matriz):
    """Calcula a razão de consistência de uma matriz"""
    n = matriz.shape[0]
    W = matriz.sum(axis=1) / matriz.sum()
    lambda_max = (matriz.dot(W) / W).mean()
    IC = (lambda_max - n) / (n - 1)
    IR = 1.25  # Para n=6
    RC = IC / IR
    return RC, W, lambda_max, IC

def verificar_ordem(W, fatores, ordem_desejada):
    """Verifica se os pesos mantém a ordem desejada"""
    pesos = {f: w for f, w in zip(fatores, W)}
    for i in range(len(ordem_desejada)-1):
        if pesos[ordem_desejada[i]] < pesos[ordem_desejada[i+1]]:
            return False
    return True

# 2. OTIMIZAÇÃO DA MATRIZ ----------------------------------------------------

print("Iniciando otimização da matriz...")

# Fatores na ordem da matriz
fatores = ordem_importancia.copy()

# Valores fixos que mantêm a hierarquia básica
valores_fixos = {
    (0,1): 2,    # LULC > DistEstradas
    (0,2): 3,    # LULC > Concentração
    (1,2): 3,    # DistEstradas > Concentração
    (2,3): 2,    # Concentração > Textura
    (3,4): 3,    # Textura > Tamanho
    (4,5): 2     # Tamanho > Declividade
}

# Elementos que serão otimizados
elementos_variaveis = {
    (0,3): 5,    # LULC vs Textura
    (0,4): 6,    # LULC vs Tamanho
    (0,5): 7,    # LULC vs Declividade
    (1,3): 4,    # DistEstradas vs Textura
    (1,4): 6,    # DistEstradas vs Tamanho
    (1,5): 6,    # DistEstradas vs Declividade
    (2,4): 5,    # Concentração vs Tamanho
    (2,5): 6,    # Concentração vs Declividade
    (3,5): 4     # Textura vs Declividade
}

# Otimização
melhor_RC = float('inf')
melhor_matriz = None
melhor_W = None
melhor_lambda_max = None
melhor_IC = None

# Criar lista de elementos para variar
chaves = list(elementos_variaveis.keys())
valores_iniciais = [elementos_variaveis[k] for k in chaves]

# Testar combinações (limitando a variações de -1, 0, +1 para eficiência)
for variacoes in tqdm(product([-1, 0, 1], repeat=len(chaves)), desc="Otimizando"):
    # Aplicar variações
    novos_valores = []
    for i, k in enumerate(chaves):
        novo_val = valores_iniciais[i] + variacoes[i]
        novo_val = max(1, min(9, novo_val))  # Manter na escala Saaty
        novos_valores.append(novo_val)

    # Construir matriz completa
    valores_completos = valores_fixos.copy()
    for i, k in enumerate(chaves):
        valores_completos[k] = novos_valores[i]

    # Construir a matriz
    mat = construir_matriz([valores_completos[(i,j)] for (i,j) in sorted(valores_completos.keys())])

    # Calcular consistência
    RC, W, lambda_max, IC = calcular_consistencia(mat)

    # Verificar ordem e consistência
    if verificar_ordem(W, fatores, ordem_importancia) and RC < melhor_RC:
        melhor_RC = RC
        melhor_matriz = mat.copy()
        melhor_W = W.copy()
        melhor_lambda_max = lambda_max
        melhor_IC = IC
        melhor_valores = valores_completos.copy()

# 3. EXPORTAÇÃO PARA EXCEL ---------------------------------------------------

print("\nPreparando arquivo Excel...")

# Criar DataFrames para exportação
df_matriz = pd.DataFrame(melhor_matriz, columns=fatores, index=fatores)
df_resultados = pd.DataFrame({
    'Fator': fatores,
    'Peso (W)': melhor_W,
    'Soma Linhas': melhor_matriz.sum(axis=1),
    'Produto Vetorial (D*W)': melhor_matriz.dot(melhor_W),
    'Lambda': melhor_matriz.dot(melhor_W) / melhor_W
})

# # Criar arquivo Excel
# with pd.ExcelWriter('matriz_comparacao_otimizada.xlsx', engine='openpyxl') as writer:
#     # Salvar matriz principal
#     df_matriz.to_excel(writer, sheet_name='Matriz Comparação', float_format="%.2f")

#     # Salvar resultados
#     df_resultados.to_excel(writer, sheet_name='Cálculos', index=False, float_format="%.4f")

#     # Acessar as planilhas para formatação
#     workbook = writer.book
#     ws_matriz = writer.sheets['Matriz Comparação']
#     ws_calc = writer.sheets['Cálculos']

#     # Formatar a planilha da matriz
#     header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
#     header_font = Font(color='FFFFFF', bold=True)

#     for col in range(1, len(fatores)+2):
#         ws_matriz.cell(row=1, column=col).fill = header_fill
#         ws_matriz.cell(row=1, column=col).font = header_font
#         ws_matriz.cell(row=col, column=1).fill = header_fill
#         ws_matriz.cell(row=col, column=1).font = header_font

#     # Formatar a planilha de cálculos
#     for col in range(1, len(df_resultados.columns)+1):
#         ws_calc.cell(row=1, column=col).fill = header_fill
#         ws_calc.cell(row=1, column=col).font = header_font

#     # Adicionar métricas de consistência
#     ws_calc['G1'] = 'Métricas de Consistência'
#     ws_calc['G2'] = f'λmax = {melhor_lambda_max:.3f}'
#     ws_calc['G3'] = f'IC = {melhor_IC:.3f}'
#     ws_calc['G4'] = f'IR = 1.25'
#     ws_calc['G5'] = f'RC = {melhor_RC:.3f}'
#     ws_calc['G6'] = 'Consistente' if melhor_RC < 0.1 else 'Inconsistente'

#     # Formatar células de métricas
#     for row in range(1, 7):
#         ws_calc.cell(row=row, column=7).font = Font(bold=True)

#     # Ajustar largura das colunas
#     for sheet in [ws_matriz, ws_calc]:
#         for column in sheet.columns:
#             max_length = 0
#             column_letter = get_column_letter(column[0].column)

#             for cell in column:
#                 try:
#                     if len(str(cell.value)) > max_length:
#                         max_length = len(str(cell.value))
#                 except:
#                     pass

#             adjusted_width = (max_length + 2) * 1.2
#             sheet.column_dimensions[column_letter].width = adjusted_width

# 4. RELATÓRIO FINAL ---------------------------------------------------------

print("\nRESULTADOS DA OTIMIZAÇÃO:")
print(f"\nMelhor Razão de Consistência encontrada: {melhor_RC:.3f}")
print("A matriz é consistente?" + (" ✅ Sim (RC < 0.1)" if melhor_RC < 0.1 else " ❌ Não (RC ≥ 0.1)"))

print("\nValores otimizados da matriz:")
for (i,j), val in sorted(melhor_valores.items()):
    original = elementos_variaveis.get((i,j), valores_fixos.get((i,j), 1))
    print(f"{fatores[i]} vs {fatores[j]}: {val} (original: {original})")

print("\nDistribuição final dos pesos:")
for f, w in zip(fatores, melhor_W):
    print(f"{f.ljust(30)}: {w:.4f}")

print("\nVerificação da ordem de importância:")
ordem_correta = True
for i in range(len(fatores)-1):
    if melhor_W[i] < melhor_W[i+1]:
        ordem_correta = False
    print(f"{fatores[i]} ({melhor_W[i]:.4f}) {'>' if melhor_W[i] >= melhor_W[i+1] else '<'} {fatores[i+1]} ({melhor_W[i+1]:.4f})")

print("\nArquivo 'matriz_comparacao_otimizada.xlsx' gerado com sucesso!")

Iniciando otimização da matriz...


Otimizando: 19683it [00:06, 2830.30it/s]


Preparando arquivo Excel...

RESULTADOS DA OTIMIZAÇÃO:

Melhor Razão de Consistência encontrada: 0.037
A matriz é consistente? ✅ Sim (RC < 0.1)

Valores otimizados da matriz:
Uso e Cobertura da Terra vs Distância à Estradas: 2 (original: 2)
Uso e Cobertura da Terra vs Concentração de Estabelecimentos: 3 (original: 3)
Uso e Cobertura da Terra vs Textura do Solo: 4 (original: 5)
Uso e Cobertura da Terra vs Tamanho de Imóveis: 7 (original: 6)
Uso e Cobertura da Terra vs Declividade: 8 (original: 7)
Distância à Estradas vs Concentração de Estabelecimentos: 3 (original: 3)
Distância à Estradas vs Textura do Solo: 3 (original: 4)
Distância à Estradas vs Tamanho de Imóveis: 6 (original: 6)
Distância à Estradas vs Declividade: 7 (original: 6)
Concentração de Estabelecimentos vs Textura do Solo: 2 (original: 2)
Concentração de Estabelecimentos vs Tamanho de Imóveis: 4 (original: 5)
Concentração de Estabelecimentos vs Declividade: 5 (original: 6)
Textura do Solo vs Tamanho de Imóveis: 3 (origin